In [ ]:
import pickle
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [ ]:
target = '2008-08_to_10'
DATA_PATH = '/content/mnt/Shareddrives/247 Project/data'
SPARSE_PKL_FILEPATH = DATA_PATH + '/{}_raw.pkl'.format(target)
SPARSE_TRAIN_FILEPATH = DATA_PATH + '/gcn_formatted_train_sparse.txt'
SPARSE_TEST_FILEPATH = DATA_PATH + '/gcn_formatted_test_sparse.txt'
PKL_FILEPATH = DATA_PATH + '/{}_densed.pkl'.format(target)
DENSE_TRAIN_FILEPATH = DATA_PATH + '/gcn_formatted_train_dense_clustered_{}.txt'.format(target)
DENSE_TEST_FILEPATH = DATA_PATH + '/gcn_formatted_test_dense_clustered_{}.txt'.format(target)
DENSE_ALL_FILEPATH = DATA_PATH + '/gcn_formatted_all_dense_clustered_{}.txt'.format(target)
CLUSTERED_PKL_FILEPATH = DATA_PATH + '/clustered_200_{}.pkl'.format(target)

# Sparse Data Formatting

In [ ]:
# data format:
# Q_ID P_ID L_ID1 L_ID2 ... L_IDn

# Q_ID --> index of all Q's
# P_IDs and L_IDs need to be combined into dictionary for unique ID values

# Replace NaN values with empty string ? or -1 to indicate none -- or just nothing in the formatted text file

In [ ]:
link_dict = {}
index_counter = 0

for p in data['P_domain']:
  if p not in link_dict.keys():
    link_dict[p] = index_counter
    index_counter += 1

# link_dict

for l in data['L_domain']:
  if not np.any(pd.isnull(l)):
    for l2 in l:
      if l2 not in link_dict.keys():
        link_dict[l2] = index_counter
        index_counter += 1

# link_dict

In [ ]:
# write lines to txt file
line_counter = 0
with open(SPARSE_TRAIN_FILEPATH, 'w') as train_doc:
  for index, info in data.iterrows():
    textLine = ""
    # append Q_ID as row ID --> this is the 'user id' for LightGCN
    #####################
    ###############
    ############## TODO: ASSUME ALL Qs ARE UNIQUE --> THIS CONNECTS TO CLUSTERING
    ############
    ##############
    ################## 
    textLine += str(index)
    # add space
    textLine += ' '
    # append P_ID and L_IDs for 'item id's for LightGCN
    P_id = info.get('P_domain')
    L_ids = info.get('L_domain')
    # lookup IDs in dictionary
    P_id_num = link_dict[P_id]
    L_id_nums = []
    if not np.any(pd.isnull(L_ids)):
      L_id_nums = [link_dict[l] for l in L_ids]
    # append to line
    textLine += str(P_id_num)
    # textLine += ' '
    L_id_string = ' '.join([str(i) for i in L_id_nums])
    if L_id_string:
      textLine += ' ' + L_id_string
    textLine += '\n'
    train_doc.write(textLine)
    # textLine
    line_counter += 1
    if line_counter % 10 == 0:
      with open(DATA_PATH + SPARSE_TEST_FILEPATH, 'a+') as test_doc:
        test_doc.write(textLine)


print("Wrote ", line_counter, " lines to ", SPARSE_TRAIN_FILEPATH)
print("Wrote ", line_counter // 10, " lines to ", SPARSE_TEST_FILEPATH)


Wrote  500000  lines to gcn_formatted_train_sparse.txt
Wrote  50000  lines to gcn_formatted_test_sparse.txt


In [ ]:
# write file of ID --> domain name mapping
import json 

with open(DATA_PATH + '/index_links_sparse.txt', 'w') as index_doc:
  index_doc.write(json.dumps(link_dict))

# Dense Data Formatting

In [ ]:
### dense data formatting
with open(PKL_FILEPATH, 'rb') as f:
  data = pickle.load(f)

data

,Q,skimed_pl
0,1 0 0 direct rendering yes single head configu...,"[code.google.com, abclinuxu.cz, boxee.tv]"
1,1 000 000 against the new facebook layout peti...,"[accelerateresults.com, www.buildingtheadaptiv..."
2,1 200 more employees right now than they need ...,"[commercialappeal.com, urbanplanet.org, nissan..."
3,1 200 more employees right now than they need ...,"[commercialappeal.com, nissannews.com, nashvil..."
4,1 200 more employees right now than they need ...,"[commercialappeal.com, nissannews.com, nashvil..."
...,...,...
7645,zunehmende spaltung zwischen arm und reich pol...,"[linkarena.com, yahoo.com, oneview.de, stern.d..."
7646,zur abgrenzung der festlandsockel in der arktis,"[sueddeutsche.de, bundestag.de, blog.de]"
7647,zwei superstars ganz pers nlich ganz pers nlic...,"[bigpoint.focus.de, partnersuche.focus.de, wei..."
7648,zwischen pest und cholera sonst werden wir sch...,"[fidelity.de, net-news-global.de, v.ppms.popul..."


In [ ]:
link_dict = {}
index_counter = 0

# link_dict

for l in data['skimed_pl']:
  for l2 in l:
    if l2 not in link_dict:
      link_dict[l2] = index_counter
      index_counter += 1


In [ ]:
print(len([l2 for l in data['skimed_pl'] for l2 in l]))  # unique links
print(len(link_dict))

39881
5405


In [ ]:
# array of how many times links are used
num_links = [0] * len(link_dict)

for l in data['skimed_pl']:
  for l2 in l:
    num_links[link_dict[l2]] += 1

# num_links
data

,Q,skimed_pl
0,1 0 0 direct rendering yes single head configu...,"[code.google.com, abclinuxu.cz, boxee.tv]"
1,1 000 000 against the new facebook layout peti...,"[accelerateresults.com, www.buildingtheadaptiv..."
2,1 200 more employees right now than they need ...,"[commercialappeal.com, urbanplanet.org, nissan..."
3,1 200 more employees right now than they need ...,"[commercialappeal.com, nissannews.com, nashvil..."
4,1 200 more employees right now than they need ...,"[commercialappeal.com, nissannews.com, nashvil..."
...,...,...
7645,zunehmende spaltung zwischen arm und reich pol...,"[linkarena.com, yahoo.com, oneview.de, stern.d..."
7646,zur abgrenzung der festlandsockel in der arktis,"[sueddeutsche.de, bundestag.de, blog.de]"
7647,zwei superstars ganz pers nlich ganz pers nlic...,"[bigpoint.focus.de, partnersuche.focus.de, wei..."
7648,zwischen pest und cholera sonst werden wir sch...,"[fidelity.de, net-news-global.de, v.ppms.popul..."


## All data

In [ ]:
# write lines to txt file
all_data = pd.DataFrame()
line_counter = 0
with open(DENSE_ALL_FILEPATH, 'w') as train_doc:
  for index, info in data.iterrows():
    textLine = ""
    # append Q_ID as row ID --> this is the 'user id' for LightGCN
    textLine += str(index)
    L_ids = info.get('skimed_pl')
    # lookup IDs in dictionary
    L_id_nums = []
    if not np.any(pd.isnull(L_ids)):
      L_id_nums = [link_dict[l] for l in L_ids]
    # textLine += ' '
    L_id_string = ' '.join([str(i) for i in L_id_nums])
    if L_id_string:
      textLine += ' ' + L_id_string
    textLine += '\n'
    train_doc.write(textLine)
    # textLine
    row = pd.DataFrame([index, L_id_nums])
    all_data = pd.concat([all_data, row], axis=1)
    line_counter += 1
    # naive implementation for test dataset
    # if line_counter % 10 == 0:
    #   with open(DATA_PATH + '/gcn_formatted_test_dense.txt', 'a+') as test_doc:
    #     test_doc.write(textLine)

all_data = all_data.transpose()
print("Wrote ", line_counter, " lines to ", DENSE_ALL_FILEPATH)
# print("Wrote ", line_counter / 10, " lines to gcn_formatted_test_dense.txt")

all_data

Wrote  7650  lines to  /content/mnt/Shareddrives/247 Project/data/gcn_formatted_all_dense_clustered_2008-08_to_10.txt


,0,1
0,0,"[0, 1, 2]"
0,1,"[3, 4, 5, 6]"
0,2,"[7, 8, 9, 10]"
0,3,"[7, 9, 10]"
0,4,"[7, 9, 10]"
...,...,...
0,7645,"[799, 800, 801, 803, 804, 805]"
0,7646,"[43, 880, 3549]"
0,7647,"[1425, 782, 1428, 1427]"
0,7648,"[2386, 2175, 2387]"


In [ ]:
# write file of ID --> domain name mapping
import json 

with open(DATA_PATH+'/index_links_dense_{}.txt'.format(target), 'w') as index_doc:
  index_doc.write(json.dumps(link_dict))

In [ ]:
# all_data = pd.read_csv(DATA_PATH+'/gcn_formatted_train_dense.txt', header=None)

all_meme_ids = pd.DataFrame(data.index)

all_links = pd.DataFrame(data['skimed_pl'])
print(all_meme_ids.shape)
print(all_links)

(7650, 1)
                                              skimed_pl
0             [code.google.com, abclinuxu.cz, boxee.tv]
1     [accelerateresults.com, www.buildingtheadaptiv...
2     [commercialappeal.com, urbanplanet.org, nissan...
3     [commercialappeal.com, nissannews.com, nashvil...
4     [commercialappeal.com, nissannews.com, nashvil...
...                                                 ...
7645  [linkarena.com, yahoo.com, oneview.de, stern.d...
7646           [sueddeutsche.de, bundestag.de, blog.de]
7647  [bigpoint.focus.de, partnersuche.focus.de, wei...
7648  [fidelity.de, net-news-global.de, v.ppms.popul...
7649  [gospodarka.gazeta.pl, wyborcza.pl, tematy-gos...

[7650 rows x 1 columns]


## Test data

In [ ]:
# train / test data split
# logic: pick random proportion of all data as test data, pick random proportion of links to use as test data
# keep test data meme and rest of links in train
np.random.seed(0)
test_data_proportion = .15
test_data_amount = int(len(all_meme_ids) * test_data_proportion)
test_data_indices = np.random.choice(all_meme_ids[0], size=test_data_amount, replace=False)
test_data_indices = np.sort(test_data_indices)

print("Test data size: ", test_data_amount)
print("Indices: ", test_data_indices)

Test data size:  1147
Indices:  [  14   29   31 ... 7629 7631 7646]


In [ ]:
test_df = pd.DataFrame()
for i in test_data_indices:
  links = all_links['skimed_pl'][i]
  n_links = len(links)
  test_links = np.random.choice(links, size = (n_links // 3), replace=False)
  row = pd.DataFrame([i, test_links])
  test_df = pd.concat([test_df, row], axis=1)

test_df = test_df.transpose()
test_df.columns = ['id', 'links']
print(test_df)

      id                                              links
0     14                                          [vh1.com]
0     29               [economist.com, osce.org, rferl.org]
0     31                           [baseball-reference.com]
0     33                             [i205.photobucket.com]
0     38  [associatedcontent.com, reddit.com, del.icio.u...
..   ...                                                ...
0   7619  [bbc.co.uk, amazon.com, ezinearticles.com, kqz...
0   7622                               [www.golivewire.com]
0   7629                                        [redlog.pl]
0   7631                                        [redlog.pl]
0   7646                                  [sueddeutsche.de]

[1147 rows x 2 columns]


In [ ]:
line_counter = 0
test_data_df = pd.DataFrame()
with open(DENSE_TEST_FILEPATH, 'w') as test_doc:
  for index, info in test_df.iterrows():
    textLine = ""
    # append Q_ID as row ID --> this is the 'user id' for LightGCN
    textLine += str(info.get('id'))
    L_ids = info.get('links')
    # lookup IDs in dictionary
    L_id_nums = []
    if not np.any(pd.isnull(L_ids)):
      L_id_nums = [link_dict[l] for l in L_ids]
    # textLine += ' '
    L_id_string = ' '.join([str(i) for i in L_id_nums])
    if L_id_string:
      textLine += ' ' + L_id_string
    textLine += '\n'
    test_doc.write(textLine)
    # textLine
    line_counter += 1
    row = pd.DataFrame([info.get('id'), L_id_nums])
    test_data_df = pd.concat([test_data_df, row], axis=1)


print("Wrote ", line_counter, " lines to ", DENSE_TEST_FILEPATH)
test_data_df = test_data_df.transpose()
test_data_df
# print("Wrote ", line_counter / 10, " lines to gcn_formatted_test_dense.tx

Wrote  1147  lines to  /content/mnt/Shareddrives/247 Project/data/gcn_formatted_test_dense_clustered_2008-08_to_10.txt


,0,1
0,14,[38]
0,29,"[82, 88, 87]"
0,31,[96]
0,33,[103]
0,38,"[131, 132, 126, 124, 138]"
...,...,...
0,7619,"[988, 167, 629, 187]"
0,7622,[2206]
0,7629,[3199]
0,7631,[3199]


## Train data

In [ ]:
# cross check with rest of data, and remove the links that have been used for test data

all_data.columns = ['id', 'links']

_, links = test_data_df.items()
_, link_arr = links

train_data_edited = pd.DataFrame()

for _, r in test_data_df.iterrows():
  # 0 -- index, 1 -- link array
  test_links = r[1]
  row_to_edit = all_data[(all_data['id'] == r[0])]
  all_links = row_to_edit['links'][0]
  train_links = np.setdiff1d(all_links, r[1])
#   train_links = all_links[::]
#   for test_link in test_links:
#       train_links.remove(test_link)
  
#   print("all links:   " + str(all_links))
#   print("test links:  " + str(test_links))
#   print("train links: " + str(train_links))

  if (len(train_links) == 0):
    print("INSIDE IF STATEMENT")
    train_links = test_links
    test_links = []

  train_row = pd.DataFrame([r[0], train_links])
  train_data_edited = pd.concat([train_data_edited, train_row], axis=1)

train_data_edited = train_data_edited.transpose()
train_data_edited.columns = ['id', 'links']
train_data_edited = train_data_edited.set_index('id')
print(train_data_edited)

                                                  links
id                                                     
14                                             [37, 39]
29                             [81, 83, 84, 85, 86, 89]
31                                             [94, 95]
33                                           [104, 105]
38    [123, 125, 127, 128, 129, 130, 133, 134, 135, ...
...                                                 ...
7619   [126, 129, 188, 278, 495, 710, 1048, 3176, 3251]
7622                                        [740, 1436]
7629                                       [3197, 4952]
7631                                 [3197, 4952, 4953]
7646                                        [880, 3549]

[1147 rows x 1 columns]


In [ ]:
train_data = all_data.set_index('id')

# train_data.head()
# train_data
train_data = train_data.transpose()
train_data.update(train_data_edited.transpose())
train_data = train_data.transpose()
train_data

,links
id,
0,"[0, 1, 2]"
1,"[3, 4, 5, 6]"
2,"[7, 8, 9, 10]"
3,"[7, 9, 10]"
4,"[7, 9, 10]"
...,...
7645,"[799, 800, 801, 803, 804, 805]"
7646,"[880, 3549]"
7647,"[1425, 782, 1428, 1427]"


In [ ]:
line_counter = 0
with open(DENSE_TRAIN_FILEPATH, 'w') as train_doc:
  for index, info in train_data.iterrows():
    textLine = ""
    # append Q_ID as row ID --> this is the 'user id' for LightGCN
    textLine += str(index)
    L_ids = info.get('links')
    # lookup IDs in dictionary
    # L_id_nums = []
    # if not np.any(pd.isnull(L_ids)):
    #   L_id_nums = [link_dict[l] for l in L_ids]
    # textLine += ' '
    L_id_string = ' '.join([str(i) for i in L_ids])
    if L_id_string:
      textLine += ' ' + L_id_string
    textLine += '\n'
    train_doc.write(textLine)
    # textLine
    line_counter += 1
    row = pd.DataFrame([info.get('id'), L_id_nums])


print("Wrote ", line_counter, " lines to ", DENSE_TRAIN_FILEPATH)


Wrote  7650  lines to  /content/mnt/Shareddrives/247 Project/data/gcn_formatted_train_dense_clustered_2008-08_to_10.txt


# Clustered Data Formatting

In [ ]:
DATA_PATH = '/content/mnt/Shareddrives/247 Project/data'
SPARSE_PKL_FILEPATH = DATA_PATH + '/intermediate_2008-08_raw.pkl'
PKL_FILEPATH = DATA_PATH + '/2008-08_densed_fixed.pkl'
DENSE_TRAIN_FILEPATH = DATA_PATH + '/gcn_formatted_train_dense_clustered.txt'
DENSE_TEST_FILEPATH = DATA_PATH + '/gcn_formatted_test_dense_clustered.txt'
DENSE_ALL_FILEPATH = DATA_PATH + '/gcn_formatted_all_dense_clustered.txt'
CLUSTERED_200_PKL_FILEPATH = DATA_PATH + '/clustered_200.pkl'
CLUSTERED_500_PKL_FILEPATH = DATA_PATH + '/clustered_500.pkl'
CLUSTERED_100_PKL_FILEPATH = DATA_PATH + '/clustered_100.pkl'
CLUSTERED_50_PKL_FILEPATH = DATA_PATH + '/clustered_50.pkl'

In [ ]:
with open(CLUSTERED_100_PKL_FILEPATH, 'rb') as f:
  data = pickle.load(f)

data

,Q,skimed_pl,embedding,cluster
0,0 g trans fats for every extra 2 percent of ca...,"[stumbleupon.com, myweb2.search.yahoo.com, fee...","[-0.0065936125, -0.020537602, 0.03334606, 0.01...",59
1,1 0 0 direct rendering yes single head configu...,"[abclinuxu.cz, code.google.com, boxee.tv]","[0.0018040149, -0.00087909284, 0.04002679, 0.0...",77
2,1 200 more employees right now than they need ...,"[nissannews.com, urbanplanet.org, nashvillecit...","[0.006087566, -0.00313511, 0.060455736, 0.0307...",62
3,1 200 more employees right now than they need ...,"[nissannews.com, urbanplanet.org, nashvillecit...","[-0.024143988, 0.00200849, 0.061202478, 0.0239...",62
4,1 200 more employees right now than they need ...,"[nissannews.com, urbanplanet.org, nashvillecit...","[-0.03216694, -0.013331371, 0.040685937, 0.019...",26
...,...,...,...,...
2687,zarbon's special suprise,"[ebay.com, slashfilm.com, jumptags.com, bcamp6...","[-0.038111545, 0.008362581, 0.008915218, -0.01...",6
2688,zoom zoom zoom engine of the year concept to r...,"[yedda.com, addthis.com, articlesbase.com]","[0.001931489, 0.001136252, 0.042400345, 0.0328...",43
2689,zrozumie e pis jest ostatnia parti z kt r mo n...,"[wiadomosci.gazeta.pl, wyborcza.pl, tematy-wia...","[-0.018571697, -0.026186584, 0.04597183, 0.017...",59
2690,zur kenntnis zu nehmen hubschraubern luftaufkl...,"[big-direkt.focus.de, blog.focus.de, p3.focus....","[-0.04226517, -0.020830031, 0.038427513, 0.015...",26


In [ ]:
data = data.groupby('cluster')

data = data.agg({'skimed_pl': 'sum'})

# data = data.apply(np.unique, axis=1)
# data = data.apply(np.squeeze)

# data = pd.DataFrame(data)
# data.columns = ['skimed_pl']
# data['skimed_pl'] = data['skimed_pl'][0]

data

,skimed_pl
cluster,
0,"[query.nytimes.com, amnesty.org, books.google...."
1,"[www.freeonline.org, freeonline.org, creativec..."
2,"[friendfeed.com, repubblica.it, splinder.com, ..."
3,"[e-prix.fr, generation-nt.com, newsvortex.net,..."
4,"[miamiherald.com, salon.com, cnn.com, photos1...."
...,...
95,"[bichenodaily.com, theskyisbig.blogspot.com, a..."
96,"[hattscrib.com, www.7chan.org, tinyurl.com, ww..."
97,"[thestar.com, gallup.com, rasmussenreports.com]"


In [ ]:
all_cluster_ids = pd.DataFrame(data.index)

all_links = pd.DataFrame(data['skimed_pl'])
all_links.index = range(100)
all_links.columns = ['links']
print(all_cluster_ids.shape)
print(all_links)

(100, 1)
                                                links
0   [query.nytimes.com, amnesty.org, books.google....
1   [www.freeonline.org, freeonline.org, creativec...
2   [friendfeed.com, repubblica.it, splinder.com, ...
3   [e-prix.fr, generation-nt.com, newsvortex.net,...
4   [miamiherald.com, salon.com, cnn.com, photos1....
..                                                ...
95  [bichenodaily.com, theskyisbig.blogspot.com, a...
96  [hattscrib.com, www.7chan.org, tinyurl.com, ww...
97    [thestar.com, gallup.com, rasmussenreports.com]
98            [fleshbot.com, gawker.com, news.com.au]
99  [i36.tinypic.com, misschairty13, www.face-pic....

[100 rows x 1 columns]


In [ ]:
link_dict = {}
index_counter = 0

for p in all_links['links']:
  for i in p:
    if i not in link_dict.keys():
      link_dict[i] = index_counter
      index_counter += 1


## All data

In [ ]:
clustered_data = pd.DataFrame()
line_counter = 0
with open('/content/mnt/Shareddrives/247 Project/data/gcn_formatted_all_dense_clustered_100.txt', 'w') as clustered_doc:
  for index, info in data.iterrows():
    textLine = ""
    # append Q_ID as row ID --> this is the 'user id' for LightGCN
    textLine += str(index)
    L_ids = info.get('skimed_pl')
    # print(info)
    # print(L_ids)
    # print(link_dict[L_ids[0]])
    # lookup IDs in dictionary
    L_id_nums = []
    if not np.any(pd.isnull(L_ids)):
      L_id_nums = [link_dict[l] for l in L_ids]
    # textLine += ' '
    L_id_string = ' '.join([str(i) for i in L_id_nums])
    if L_id_string:
      textLine += ' ' + L_id_string
    textLine += '\n'
    clustered_doc.write(textLine)
    # textLine
    row = pd.DataFrame([index, L_id_nums])
    clustered_data = pd.concat([clustered_data, row], axis=1)
    line_counter += 1

clustered_data = clustered_data.transpose()
print("Wrote ", line_counter, " lines to gcn_formatted_all_dense_clustered_100.txt")
# print("Wrote ", line_counter / 10, " lines to gcn_formatted_test_dense.txt")

clustered_data

Wrote  100  lines to gcn_formatted_all_dense_100.txt


,0,1
0,0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
0,1,"[46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5..."
0,2,"[82, 83, 84, 85, 86, 87, 88, 84, 89, 85, 90, 4..."
0,3,"[187, 188, 189, 190, 191, 192, 193, 194, 195, ..."
0,4,"[246, 247, 248, 249, 250, 251, 252, 253, 254, ..."
...,...,...
0,95,"[1733, 1734, 1735, 1736, 1737, 1738, 1739, 174..."
0,96,"[117, 668, 175, 119, 277]"
0,97,"[788, 1812, 1650]"
0,98,"[1238, 170, 711]"


## Test data

In [ ]:
# train / test data split

# logic: pick random proportion of all data as test data, pick random proportion of links to use as test data
# keep test data meme and rest of links in train

test_data_proportion = .15
test_data_amount = int(len(clustered_data) * test_data_proportion)
test_data_indices = np.random.choice(clustered_data[0], size=test_data_amount, replace=False)
test_data_indices = np.sort(test_data_indices)

print("Test data size: ", test_data_amount)
print("Indices: ", test_data_indices)

Test data size:  15
Indices:  [4 9 15 23 33 37 48 58 61 65 67 69 77 89 97]


In [ ]:
test_df = pd.DataFrame()
for i in test_data_indices:
  links = all_links['links'][i]
  n_links = len(links)
  test_links = np.random.choice(links, size = (n_links // 3), replace=False)
  row = pd.DataFrame([i, test_links])
  test_df = pd.concat([test_df, row], axis=1)
  
test_df = test_df.transpose()
test_df.columns = ['id', 'links']

test_df

,id,links
0,4,"[kurse.focus.de, www.hattscrib.com, bookmarks...."
0,9,"[www.epiccarnival.com, foxnews.com, publishedm..."
0,15,"[www.squidoo.com, amazon.com]"
0,23,"[jumptags.com, i36.tinypic.com, news.softpedia..."
0,33,[spurl.net]
0,37,[wetpaintcentral.com]
0,48,"[news.google.de, rss.focus.de, tomorrow-focus.de]"
0,58,[influentialblogger.net]
0,61,[blog.washingtonpost.com]
0,65,[japantimes.com]


In [ ]:
line_counter = 0
test_data_df = pd.DataFrame()
with open('/content/mnt/Shareddrives/247 Project/data/gcn_formatted_test_dense_clustered_100.txt', 'w') as test_doc:
  for index, info in test_df.iterrows():
    textLine = ""
    # append Q_ID as row ID --> this is the 'user id' for LightGCN
    textLine += str(info.get('id'))

    L_ids = info.get('links')
    # print(L_ids)
    # lookup IDs in dictionary
    L_id_nums = []
    if not np.any(pd.isnull(L_ids)):
      L_id_nums = [link_dict[l] for l in L_ids]
    # textLine += ' '
    L_id_string = ' '.join([str(i) for i in L_id_nums])
    if L_id_string:
      textLine += ' ' + L_id_string
    textLine += '\n'
    test_doc.write(textLine)
    # textLine
    line_counter += 1
    row = pd.DataFrame([info.get('id'), L_id_nums])
    test_data_df = pd.concat([test_data_df, row], axis=1)


print("Wrote ", line_counter, " lines to gcn_formatted_test_dense_clustered_100.txt")
test_data_df = test_data_df.transpose()
test_data_df
# print("Wrote ", line_counter / 10, " lines to gcn_formatted_test_dense.tx

Wrote  15  lines to gcn_formatted_test_dense_clustered_100.txt


,0,1
0,4,"[235, 119, 251, 360, 576, 371, 564, 390, 643, ..."
0,9,"[803, 370, 804, 417, 793, 812, 785, 796, 171, ..."
0,15,"[820, 295]"
0,23,"[154, 118, 63, 1333, 118, 34, 138, 1414, 230, ..."
0,33,[174]
0,37,[1466]
0,48,"[620, 229, 231]"
0,58,[152]
0,61,[480]
0,65,[553]


## Train data

In [ ]:
# cross check with rest of data, and remove the links that have been used for test data

clustered_data.columns = ['id', 'links']

index, links = test_data_df.items()
id_num, link_arr = links

train_data = pd.DataFrame()

for i, r in test_data_df.iterrows():
  # 0 -- index, 1 -- link array
  # print(r)
  row_to_edit = clustered_data[(clustered_data['id'] == r[0])]
  links_left = np.setdiff1d(row_to_edit['links'][0], r[1])
  if (len(links_left) == 0):
    print("INSIDE IF STATEMENT")
    links_left = r[1]
    print(links_left)
  # print("row to edit: ", row_to_edit, " r: ", r[1])
  # print(links_left)
  train_row = pd.DataFrame([r[0], links_left])
  train_data = pd.concat([train_data, train_row], axis=1)

train_data = train_data.transpose()
train_data.columns = ['id', 'links']
train_data = train_data.set_index('id')
clustered_data = clustered_data.set_index('id')

# train_data.head()
# train_data
clustered_data.update(train_data)

clustered_data

,links
id,
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,"[46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 5..."
2,"[82, 83, 84, 85, 86, 87, 88, 84, 89, 85, 90, 4..."
3,"[187, 188, 189, 190, 191, 192, 193, 194, 195, ..."
4,"[2, 12, 20, 36, 37, 38, 40, 42, 43, 44, 46, 47..."
...,...
95,"[1733, 1734, 1735, 1736, 1737, 1738, 1739, 174..."
96,"[117, 668, 175, 119, 277]"
97,"[788, 1812]"


In [ ]:
line_counter = 0
with open('/content/mnt/Shareddrives/247 Project/data/gcn_formatted_train_dense_clustered_100.txt', 'w') as train_doc:
  for index, info in clustered_data.iterrows():
    textLine = ""
    # append Q_ID as row ID --> this is the 'user id' for LightGCN
    textLine += str(index)
    L_ids = info.get('links')
    # print(L_ids)
    # lookup IDs in dictionary
    # L_id_nums = []
    # if not np.any(pd.isnull(L_ids)):
    #   L_id_nums = [link_dict[l] for l in L_ids]
    # textLine += ' '
    L_id_string = ' '.join([str(i) for i in L_ids])
    if L_id_string:
      textLine += ' ' + L_id_string
    textLine += '\n'
    train_doc.write(textLine)
    # textLine
    line_counter += 1
    # row = pd.DataFrame([info.get('id'), L_id_nums])


print("Wrote ", line_counter, " lines to gcn_formatted_train_dense_clustered_100.txt")


Wrote  100  lines to gcn_formatted_train_dense_clustered_100.txt
